In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
# config 설정(Delta Lake 라이브러리 추가)
# getOrCreate 기존 SparkSession이 있으면 그것을 반환, 그렇지 않으면 새롭게 생성

your 131072x1 screen size is bogus. expect trouble


24/04/08 13:41:34 WARN Utils: Your hostname, Kimbugeon resolves to a loopback address: 127.0.1.1; using 172.22.182.179 instead (on interface eth0)
24/04/08 13:41:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/bugeon/deltalakeenv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bugeon/.ivy2/cache
The jars for the packages stored in: /home/bugeon/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7cd7cc1d-31e5-463e-a84d-50d243af29d7;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 148ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

24/04/08 13:41:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

In [11]:
data = spark.range(0, 5)
data.write.format("delta").save("./delta-table")

In [12]:
df = spark.read.format("delta").load("./delta-table")
df.show()

+---+
| id|
+---+
|  3|
|  1|
|  4|
|  0|
|  2|
+---+



In [13]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("./delta-table")

In [14]:
df = spark.read.format("delta").load("./delta-table")
df.show()

+---+
| id|
+---+
|  7|
|  5|
|  8|
|  9|
|  6|
+---+



In [15]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "./delta-table")

# 모든 짝수 값에 100을 더해 업데이트
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# 모든 짝수 값 삭제
deltaTable.delete(condition = expr("id % 2 == 0"))

# 새 데이터 삽입(병합)
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

24/04/04 22:47:06 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
24/04/04 22:47:06 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
24/04/04 22:47:06 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [16]:
strmdf = spark.readStream.format("rate").load()
stream = strmdf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "delta-table/checkpoint").start("./delta-table")


24/04/04 23:15:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [17]:
stream = spark.readStream.format("delta").load("./delta-table").writeStream.format("console").start()


24/04/04 23:15:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c659279d-f73b-43f9-9fe5-011aa9a3bf4e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/04 23:15:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+---+
| id|
+---+
|  3|
+---+

-------------------------------------------
Batch: 2
-------------------------------------------
+---+
| id|
+---+
|  4|
+---+

-------------------------------------------
Batch: 3
-------------------------------------------
+---+
| id|
+---+
|  5|
+---+

-------------------------------------------
Batch: 4
-------------------------------------------
+---+
| id|
+---+
|  6|
+---+

-------------------------------------------
Batch: 5
-------------------------------------------
+---+
| id|
+---+
|  7|
+---+

-------------------------------------------
Batch: 6
----------------------------------------

In [18]:
stream.stop()

In [9]:
spark.sql("CREATE DATABASE IF NOT EXISTS delta")


DataFrame[]

In [5]:
from delta.tables import *
from pyspark.sql.functions import *

"""
DeltaTable.createIfNotExists(spark) \
  .tableName("default.people10m") \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .execute()
  """

DeltaTable.createOrReplace(spark) \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .property("description", "table with people data") \
  .location("/tmp/delta/people10m") \
  .execute()


In [24]:
people = spark.read.format("delta").load("/tmp/delta/people10m")
people.show()

+---+---------+----------+--------+------+---------+---+------+
| id|firstName|middleName|lastName|gender|birthDate|ssn|salary|
+---+---------+----------+--------+------+---------+---+------+
+---+---------+----------+--------+------+---------+---+------+



In [13]:
DeltaTable.create(spark) \
  .tableName("default.events") \
  .addColumn("eventId", "BIGINT") \
  .addColumn("data", "STRING") \
  .addColumn("eventType", "STRING") \
  .addColumn("eventTime", "TIMESTAMP") \
  .addColumn("eventDate", "DATE", generatedAlwaysAs="CAST(eventTime AS DATE)") \
  .partitionedBy("eventType", "eventDate") \
  .execute()

In [21]:
event = spark.sql('SELECT * FROM default.events WHERE eventTime >= "2020-10-01 00:00:00" AND eventTime <= "2020-10-01 12:00:00"')
event.show()

+-------+----+---------+---------+---------+
|eventId|data|eventType|eventTime|eventDate|
+-------+----+---------+---------+---------+
+-------+----+---------+---------+---------+



In [37]:
# 기존 delta 테이블에 새 데이터를 일관되게 처리하기 위해 append 모드 사용
people.write.format("delta").mode("append").save("/tmp/delta/people10m")


In [38]:
# 테이블의 모든 데이터를 원자적으로 바꾸려면 overwrite 모드 사용
people.write.format("delta").mode("overwrite").save("/tmp/delta/people10m")


In [9]:
from pyspark.sql.functions import col

# 기존 Delta 테이블 읽어옴
people = spark.read.format("delta").load("/tmp/delta/people10m")

# 새로운 데이터를 포함하는 DataFrame을 생성
people2 = spark.createDataFrame([
    (3, "Emily", "Johnson", "Female", "man", "1990-03-20", "555-12-3456", 70000),
    (4, "Michael", "Williams", "Male", "wuman", "1982-07-10", "888-99-0000", 80000)
], ["id", "firstName", "middleName", "lastName", "gender", "birthDate", "ssn", "salary"])

# 컬럼의 데이터 타입을 명시(타입이 안맞다고 오류 시 유용함)
people2 = people2.withColumn("id", col("id").cast("int"))
people2 = people2.withColumn("birthDate", col("birthDate").cast("TIMESTAMP"))
people2 = people2.withColumn("salary", col("salary").cast("int"))

# 기존 Delta 테이블과 새로운 데이터를 결합
people3 = people.union(people2)

# 결합된 데이터를 다시 Delta 테이블에 저장
people3.write.format("delta").mode("overwrite").save("/tmp/delta/people10m")


In [10]:
people3.show()

+---+---------+----------+--------+------+-------------------+-----------+------+
| id|firstName|middleName|lastName|gender|          birthDate|        ssn|salary|
+---+---------+----------+--------+------+-------------------+-----------+------+
|  4|  Michael|  Williams|    Male| wuman|1982-07-10 00:00:00|888-99-0000| 80000|
|  3|    Emily|   Johnson|  Female|   man|1990-03-20 00:00:00|555-12-3456| 70000|
|  3|    Emily|   Johnson|  Female|   man|1990-03-20 00:00:00|555-12-3456| 70000|
|  4|  Michael|  Williams|    Male| wuman|1982-07-10 00:00:00|888-99-0000| 80000|
+---+---------+----------+--------+------+-------------------+-----------+------+

